In [1]:
import os
import torch
from torchvision.datasets.utils import download_url
from torchtext import data
from torchtext import datasets
import pandas as pd
import numpy as np

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
%matplotlib inline
import time
from datetime import datetime
import random
from sklearn.metrics import f1_score

In [2]:
import models

In [3]:
SEED = 1
random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = False

### Load data

In [5]:
# OR load full data, if produced. 435 MB. NOT in Github currently. 
# You can produce this in about 30 min with preprocess_data.ipynb
reuters = pd.read_pickle('input/reuters_all.pkl')

In [6]:
print(len(reuters))
reuters[0:2]

299773


,codes,headline,text
0,"[C18, C181, CCAT]",Eureko is latest suitor for French insurer GAN.,"\nEureko, an alliance of six European financia..."
1,"[G15, GCAT]",Reuter EC Report Long-Term Diary for July 28 -...,\n****\nHIGHLIGHTS\n****\nLUXEMBOURG - Luxembo...


In [7]:
# read classcodes
classcodes= pd.read_csv('input/classcodes.csv')
print(len(classcodes))
#classcodes[0:12]

# add index field to DataFrame
classcodes = classcodes.reset_index()
# Create dictionary index/int to classcode and classcode to int
itocode = dict(zip(classcodes.index, classcodes.Code))
codetoi = dict(zip(classcodes.Code, classcodes.index))
def listToInt(mylist):
    return [codetoi[item] for item in mylist]

reuters['codes'] = [listToInt(codelist) for codelist in reuters.codes]
reuters[0:3]
# Multihot, for single list - one row
def multihot(tags):
    return [1 if tag in tags else 0 for tag in taglist]

# list of classes, 126 int: [0...125]
taglist = list(classcodes.index)
Y_hot = [multihot(claslist) for claslist in reuters.codes]
reuters['codes'] = Y_hot

126


In [8]:
print(len(reuters))
reuters[0:2]

299773


,codes,headline,text
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Eureko is latest suitor for French insurer GAN.,"\nEureko, an alliance of six European financia..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Reuter EC Report Long-Term Diary for July 28 -...,\n****\nHIGHLIGHTS\n****\nLUXEMBOURG - Luxembo...


### Using DataSet


Example in:
https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/A%20-%20Using%20TorchText%20with%20Your%20Own%20Datasets.ipynb

In [9]:
#Define the Fields
TEXT = data.Field()
HEADLINE = data.Field()
LABELS = data.LabelField(sequential=False, use_vocab=False)

In [10]:
# Put data in random order
SEED = 1
random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
idx = np.random.permutation(len(reuters))
reuters = reuters.iloc[idx]

In [11]:
# split it
size = len(reuters)
train_size = int(0.7*size)
test_size = int(0.85*size)
# val is rest

train = reuters[0: train_size]
val = reuters[train_size : test_size]
test = reuters[test_size : size]

train.to_json('input/train.json', orient='records', lines=True)
val.to_json('input/val.json', orient='records', lines=True)
test.to_json('input/test.json', orient='records', lines=True)

### For final training only: use all training data
dev/test sets are now part of same training data, so they will show too high score.
Stop training at same nr epochs found good with separete train/dev/set.

In [12]:
# For final training only, use all data
# split it
size = len(reuters)

# After finding model, higher training data, run same nr of epochs
#train_size = int(0.95*size)
train_size = size # use all data, will overfit dev set but its ok
test_size = int(0.97*size)
# val is rest

train = reuters[0: train_size]
#val = reuters[train_size : test_size]
val = reuters[test_size : size]
test = reuters[test_size : size] # same as val, not needed

train.to_json('input/train.json', orient='records', lines=True)
val.to_json('input/val.json', orient='records', lines=True)
test.to_json('input/test.json', orient='records', lines=True)

In [13]:
# Tell torchText which Fields to apply to which json elements
fields = {'headline': ('h', HEADLINE), 'text': ('t', TEXT), 'codes': ('l', LABELS)}
fields

{'headline': ('h', <torchtext.data.field.Field at 0x7fe5c90ce7b8>),
 'text': ('t', <torchtext.data.field.Field at 0x7fe5c90ce6d8>),
 'codes': ('l', <torchtext.data.field.LabelField at 0x7fe5c90ce748>)}

In [14]:
# Create dataset (TabularDataset)
train_data, valid_data, test_data = data.TabularDataset.splits(
                                        path = 'input',
                                        train = 'train.json',
                                        validation = 'val.json',
                                        test = 'test.json',
                                        format = 'json',
                                        fields = fields
)

In [15]:
BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE,
    device=device,
    sort_key= lambda x: len(x.t)
)

In [16]:
# GLOVE
TEXT.build_vocab(train_data, max_size=25000, vectors="glove.6B.50d")
HEADLINE.build_vocab(train)
LABELS.build_vocab(train)

pretrained_embeddings = TEXT.vocab.vectors
print(pretrained_embeddings.shape)

.vector_cache/glove.6B.zip: 862MB [03:52, 3.71MB/s]                               
100%|█████████▉| 398491/400000 [00:11<00:00, 33411.15it/s]

torch.Size([25002, 50])


### Print some info

In [17]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')
print('------------')
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABELS vocabulary: {len(LABELS.vocab)}")

Number of training examples: 299773
Number of validation examples: 8994
Number of testing examples: 8994
------------
Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABELS vocabulary: 3


In [18]:
print(TEXT.vocab.freqs.most_common(10))
print('-----')
print(TEXT.vocab.itos[:10])
print(LABELS.vocab.stoi)

[('the', 3112856), ('of', 1664693), ('to', 1661936), ('in', 1321409), ('a', 1238399), ('and', 1238227), ('on', 762926), ('said', 591160), ('for', 579301), ('The', 491337)]
-----
['<unk>', '<pad>', 'the', 'of', 'to', 'in', 'a', 'and', 'on', 'said']
defaultdict(<function _default_unk_index at 0x7fe6505d37b8>, {'codes': 0, 'headline': 1, 'text': 2})


### F1 accuracy

The results will be ranked according to the highest micro-averaged F1 score. 
This will be calculated using the f1_score function found in scikit-learn, using a command like 
f1_score(y_true, y_pred, average='micro') where y_true is the matrix with the ground truth, and y_pred 
the predicted output. Both matrices are binary, a 1 in row i and column j means that the image/document
i contains the label j.

Scikit:  Micro-average in F1-score
 
'micro':
    Calculate metrics globally by counting the total true positives, false negatives and false positives.

In [19]:
# f1 score for BATCH
from sklearn.metrics import f1_score
def f1_accuracy(preds, y):
    """
    Returns f1 accuracy from sklearn
    """
    #round predictions to the closest integer
    #rounded_preds = torch.round(torch.sigmoid(preds))
    rounded_preds = torch.round(preds)
    
    preds_cpu = rounded_preds.cpu().data.numpy()
    y_cpu = y.cpu().data.numpy()
    f1 = f1_score(y_cpu, preds_cpu, average='micro')
    return f1 

In [20]:
def f1_own_accuracy(preds, y):
    '''for micro-average
    Returns counts of true_pos, false_pos and false_negative.
    For counting precision, recall and F1 globally
    precision = true_pos / (true_pos + false_pos)
    recall = true_pos / (true_pos + false_neg)
    '''
    #round predictions to the closest integer
    #rounded_preds = torch.round(torch.sigmoid(preds))
    rounded_preds = torch.round(preds)
    
    preds = rounded_preds.cpu().data.numpy()
    y = y.cpu().data.numpy()
        
    # True positive
    tpos = np.sum(np.logical_and(preds == 1, y == 1))
 
    # True negative
    #tneg = np.sum(np.logical_and(preds == 0, y == 0))
 
    # False positive
    fpos = np.sum(np.logical_and(preds == 1, y == 0))
 
    # False negative
    fneg = np.sum(np.logical_and(preds == 0, y == 1))

    return tpos, fpos, fneg

In [21]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    #rounded_preds = torch.round(torch.sigmoid(preds))
    rounded_preds = torch.round(preds)
    
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

In [22]:
# F1 version
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    
    epoch_tpos = 0
    epoch_fpos = 0
    epoch_fneg = 0
    
    model.train()
    
    for batch in iterator:
        if batch.t.shape[1] != BATCH_SIZE:
              continue
        
        optimizer.zero_grad()
                
        predictions = model(torch.cat((batch.t, batch.h))).squeeze(1)
        
        loss = criterion(predictions, batch.l.float())
        
        tpos, fpos, fneg = f1_own_accuracy(predictions, batch.l.float())
        epoch_tpos += tpos
        epoch_fpos += fpos
        epoch_fneg += fneg
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        #epoch_acc += acc.item()
    
    # Counted f1-score is Micro-average version
    # avoid div by zero with epsilon. 
    # F1 for 0 - no labels is not defined, but here we give it score 1
    eps = 1e-7
    epoch_precision = epoch_tpos / (epoch_tpos + epoch_fpos +eps )
    epoch_recall = epoch_tpos / (epoch_tpos + epoch_fneg +eps)
    epoch_f1 = 2* (  ((epoch_precision * epoch_recall)+eps) / (epoch_precision + epoch_recall +2*eps))
    
    # zero true_positive can cause F1=1, (because only 2*(epsilon / 2epsilon) remains, leading to 1)
    # fix it to zero
    if (epoch_precision==0 and epoch_recall==0):
        epoch_f1 = 0
    
    return epoch_loss / len(iterator), epoch_precision, epoch_recall, epoch_f1

In [23]:
# F1 version
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    #epoch_acc = 0
    epoch_tpos = 0
    epoch_fpos = 0
    epoch_fneg = 0    
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            if batch.t.shape[1] != BATCH_SIZE:
                continue

            predictions = model(torch.cat((batch.t, batch.h))).squeeze(1)
            
            loss = criterion(predictions, batch.l.float())
            
            tpos, fpos, fneg = f1_own_accuracy(predictions, batch.l.float())
            epoch_tpos += tpos
            epoch_fpos += fpos
            epoch_fneg += fneg            

            epoch_loss += loss.item()
            #epoch_acc += acc.item()

    # avoid div by zero with epsilon
    eps = 1e-7
    epoch_precision = epoch_tpos / (epoch_tpos + epoch_fpos +eps)
    epoch_recall = epoch_tpos / (epoch_tpos + epoch_fneg +eps)
    epoch_f1 = 2* (  ((epoch_precision * epoch_recall)+eps) / (epoch_precision + epoch_recall +2*eps))          
    
    # zero true_positive can cause F1=1, (because only 2*(epsilon / 2epsilon) remains, leading to 1)
    # fix it to zero
    if (epoch_precision==0 and epoch_recall==0):
        epoch_f1 = 0
    
    return epoch_loss / len(iterator), epoch_precision, epoch_recall, epoch_f1

In [24]:
def predict(model, iterator):
        
    model.eval()
    preds = []
    
    with torch.no_grad():
    
        for batch in iterator:
            #if batch.t.shape[1] != BATCH_SIZE:
            #    continue

            predictions = model(torch.cat((batch.t, batch.h))).squeeze(1)
            predictions = torch.round(predictions)
            
            cpu_pred = predictions.cpu()
            result = cpu_pred.data.numpy()            
            # unpack the #batch nr of predictions and add individually
            for item in result:
                preds.append(item)
    
    return preds

In [78]:
def predict_prob(model, iterator):
    ''' Dont round predictions to 0 and 1, but keep as probability: float between 0 and 1.
    
    '''
        
    model.eval()
    preds = []
    
    with torch.no_grad():
    
        for batch in iterator:
            #if batch.t.shape[1] != BATCH_SIZE:
            #    continue

            predictions = model(torch.cat((batch.t, batch.h))).squeeze(1)
            # ROUNDING AWAY
            #predictions = torch.round(predictions)
            
            cpu_pred = predictions.cpu()
            result = cpu_pred.data.numpy()            
            # unpack the #batch nr of predictions and add individually
            for item in result:
                preds.append(item)
    
    return preds

### Load model

In [25]:
# Common
INPUT_DIM = len(TEXT.vocab) # 25002
EMBEDDING_DIM = 50
OUTPUT_DIM = 126

### Models to try

Add wanted models to list, then train them all

In [26]:
# place to put model definitions
try_models = []
try_descs = []
try_epochs = []

### Try for best accuracy

In [27]:
# CNN with high nr of filters
N_FILTERS = 300
FILTER_SIZES = [3,5,7]
DROPOUT = 0.5
N_EPOCHS = 6 # full data, try 6
#N_EPOCHS = 1 # temporary

model = models.CNN2(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT)

# 14 epochs gave f1 0.84
desc = "CNN 300x3,5,7 filters"

try_models.append(model)
try_descs.append(desc)
try_epochs.append(N_EPOCHS)

In [28]:
# COMMON
def init_models():
    ''' init global parameters'''
    global model
    global optimizer
    global criterion
    global train_losses
    global train_f1s
    global val_losses
    global val_f1s
    global times
    global SEED
    global torch
    
    model = model.to(device)
    model.embedding.weight.data.copy_(pretrained_embeddings)

    optimizer = optim.Adam(model.parameters())

    #criterion = nn.BCEWithLogitsLoss()
    criterion = nn.BCELoss()
    criterion = criterion.to(device)

    # Reset 
    train_losses=[]
    train_f1s=[]
    val_losses=[]
    val_f1s=[]
    times=[]

    # If want repeatability
    SEED = 1
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    
    # Faster
    torch.backends.cudnn.deterministic = False

In [29]:
#model

In [30]:
#optimizer # default Adam

In [31]:
#optimizer = optim.Adam(model.parameters(), lr=1e-3, betas=(0.9, 0.99))

In [32]:
try_descs

['CNN 300x3,5,7 filters']

### Train all models

In [33]:
# Store results of training
results = pd.DataFrame()

for i in range(len(try_models)):
    
    model = try_models[i]
    desc = try_descs[i]
    epochs = try_epochs[i]
    print(f'Training model: {desc}')

    # init global parameters
    init_models()
    model_name = type(model).__name__

    #N_EPOCHS = 3
    N_EPOCHS = epochs

    for epoch in range(N_EPOCHS):

        start = time.time()
        train_loss, train_precision, train_recall, train_f1 = train(model, train_iterator, optimizer, criterion)
        end = time.time(); elapsed = end-start
    
        valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(model, valid_iterator, criterion)
    
        times.append(elapsed) 
        train_losses.append(train_loss); train_f1s.append(train_f1)
        val_losses.append(valid_loss); val_f1s.append(valid_f1)
    
        print(f'| Ep:{epoch+1:02} |Tr Loss:{train_loss:.3f} |Prec:{train_precision:.3f} |Rec:{train_recall:.3f} |f1:{train_f1:.3f} |Val Loss:{valid_loss:.3f} |prec:{valid_precision:.3f} |rec:{valid_recall:.3f} |f1:{valid_f1:.3f} |')        


    #Test F1-score
    test_loss, test_precision, test_recall, test_f1 = evaluate(model, test_iterator, criterion)
    print(f'| Test Loss: {test_loss:.3f} ||Prec:{test_precision:.3f} |Rec:{test_recall:.3f} |Test F1:{test_f1: .3f} |')        
        
    # Store results
    times_cumul = pd.Series(times).cumsum() # cumulative time    
    results = results.append({'Model': model_name,
                          'Desc' : desc,
                          'Test_f1': test_f1,
                          'Time': times_cumul,
                          'Train_loss': pd.DataFrame({'Train_loss':train_losses}), 
                          'Train_f1': pd.DataFrame({'Train_f1':train_f1s}), 
                          'Val_loss': pd.DataFrame({'Val_loss':val_losses}), 
                          'Val_f1': pd.DataFrame({'Val_f1':val_f1s}),
                                         }, ignore_index=True)

    print(" ")
    print(f" Total time used for training: {sum(times)} s ##########")
    print(" ")

Training model: CNN 300x3,5,7 filters


/home/max/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
100%|█████████▉| 398491/400000 [00:30<00:00, 33411.15it/s]

| Ep:01 |Tr Loss:0.035 |Prec:0.857 |Rec:0.662 |f1:0.747 |Val Loss:0.026 |prec:0.851 |rec:0.811 |f1:0.830 |
| Ep:02 |Tr Loss:0.025 |Prec:0.882 |Rec:0.762 |f1:0.818 |Val Loss:0.026 |prec:0.827 |rec:0.860 |f1:0.843 |
| Ep:03 |Tr Loss:0.024 |Prec:0.889 |Rec:0.783 |f1:0.832 |Val Loss:0.023 |prec:0.848 |rec:0.865 |f1:0.857 |
| Ep:04 |Tr Loss:0.022 |Prec:0.893 |Rec:0.794 |f1:0.841 |Val Loss:0.022 |prec:0.852 |rec:0.875 |f1:0.864 |
| Ep:05 |Tr Loss:0.021 |Prec:0.896 |Rec:0.803 |f1:0.847 |Val Loss:0.020 |prec:0.868 |rec:0.877 |f1:0.873 |
| Ep:06 |Tr Loss:0.021 |Prec:0.899 |Rec:0.810 |f1:0.852 |Val Loss:0.019 |prec:0.881 |rec:0.877 |f1:0.879 |
| Test Loss: 0.019 ||Prec:0.881 |Rec:0.877 |Test F1: 0.879 |
 
 Total time used for training: 2704.6476027965546 s ##########
 


In [34]:
results   

,Desc,Model,Test_f1,Time,Train_f1,Train_loss,Val_f1,Val_loss
0,"CNN 300x3,5,7 filters",CNN2,0.879047,0 449.065401 1 896.267396 2 1346.94...,Train_f1 0 0.747033 1 0.817532 2 0.83243...,Train_loss 0 0.034770 1 0.025388 2 ...,Val_f1 0 0.830396 1 0.843459 2 0.85650...,Val_loss 0 0.026226 1 0.025973 2 0.02328...


### Save training results - score and time

In [36]:
datetime_string = datetime.now().strftime("%Y%m%d-%H%M")
print(datetime_string)
results.to_pickle('model_results/results_CNN_all_traindata'+datetime_string+'.pkl')

20190218-1449


### Save the model

For this version create the model object with same paramters as when training. Then load weights.
This version saves also gradients etc.

In [37]:
torch.save(model.state_dict(), 'models/model_DICT_CNN300_f1_all_traindata_0_853_train.pt')
#torch.save(model, 'filename.pt')

In [38]:
# Whole model
torch.save(model, 'models/model_WHOLE_CNN300_f1_all_traindata_0_853_train.pt')

### Optional - Load Model

In [54]:
# If Whole model
model = torch.load('models/model_WHOLE_CNN300_f1_all_traindata_0_853_train.pt')

In [ ]:
#Only Dict / weights

In [ ]:
# USE same creation as when model was trained
#model2 = RNN(input_dim=25002, embedding_dim=50, hidden_dim=256, output_dim=126)

In [28]:
model.load_state_dict(torch.load('model_DICT_CNN300_f1_all_traindata_0_853_train.pt'))                 

In [29]:
model.eval()

CNN2(
  (embedding): Embedding(25002, 50)
  (convs): ModuleList(
    (0): Conv2d(1, 300, kernel_size=(3, 50), stride=(1, 1))
    (1): Conv2d(1, 300, kernel_size=(5, 50), stride=(1, 1))
    (2): Conv2d(1, 300, kernel_size=(7, 50), stride=(1, 1))
  )
  (fc): Linear(in_features=900, out_features=126, bias=True)
  (dropout): Dropout(p=0.5)
)

In [48]:
criterion = nn.BCELoss()
criterion = criterion.to(device)
model.to(device)

CNN2(
  (embedding): Embedding(25002, 50)
  (convs): ModuleList(
    (0): Conv2d(1, 300, kernel_size=(3, 50), stride=(1, 1))
    (1): Conv2d(1, 300, kernel_size=(5, 50), stride=(1, 1))
    (2): Conv2d(1, 300, kernel_size=(7, 50), stride=(1, 1))
  )
  (fc): Linear(in_features=900, out_features=126, bias=True)
  (dropout): Dropout(p=0.5)
)

## Test it on test data

In [39]:
test_loss, test_precision, test_recall, test_f1 = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} ||Prec:{test_precision:.3f} |Rec:{test_recall:.3f} |Test F1:{test_f1: .3f} |')

/home/max/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


| Test Loss: 0.019 ||Prec:0.881 |Rec:0.877 |Test F1: 0.879 |


### Optional -  Extra test using training sets test-data in place of new data, so we have known labels to confirm result

In [40]:
# New data dataset
new_dataset1 = data.TabularDataset(
                                        path = 'input/test.json',
                                        format = 'json',
                                        #fields = fields_predict
                                        fields = fields
)

In [43]:
# Print out example
ex = new_dataset1[1]
ex.t[0:10]

['Punjab',
 'varieties',
 'remained',
 'steady',
 'at',
 'the',
 'improved',
 'level',
 'but',
 'Gujarat']

In [44]:
# Print out example
ex.h

['India', 'cotton', 'mixed,', 'dealers', 'seek', 'Gujarat', 'varieties.']

In [57]:
BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

new_iterator1 = data.Iterator(
    dataset=new_dataset1, 
    batch_size=BATCH_SIZE,
    device=device,
    sort_key= lambda x: len(x.t),
    #sort_key = None,
    train=False,
    sort=False
)

In [148]:
preds = predict(model, new_iterator1)

# moved inside predict()
#rounded_preds = torch.round(preds)   
#preds_cpu = rounded_preds.cpu().data.numpy()

/home/max/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [67]:
preds = np.asarray(preds)

In [68]:
preds_pd = pd.DataFrame(preds)
preds_pd.to_csv('output/predictions_trainset_test.csv', header=False, index=False)

In [69]:
test = pd.read_json('input/test.json', orient='records', lines=True)

In [70]:
# Ground truth, from pandas DataFrame
# Each is a 126 item list in a DataFrame
# Trun into 2-dim np.array

# explode the list into columns
temp = test['codes'].apply(pd.Series)
# turn to np.array
y_true = np.asarray(temp)

In [71]:
from sklearn.metrics import f1_score

f1_score(y_true,
         preds,
         average='micro')

0.8777632571922913

#### Was 0.16082617373947755 with sort=None. It still sorted test set. Needs to be sort=False

In [64]:
test_loss, test_precision, test_recall, test_f1 = evaluate(model, new_iterator1, criterion)

print(f'| Test Loss: {test_loss:.3f} ||Prec:{test_precision:.3f} |Rec:{test_recall:.3f} |Test F1:{test_f1: .3f} |')

/home/max/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


| Test Loss: 0.016 ||Prec:0.933 |Rec:0.829 |Test F1: 0.878 |


### Use Delta to find best threshold to split 0 and 1

In [ ]:
# https://www.kaggle.com/ziliwang/baseline-pytorch-bilstm
def bestThresshold(train_y,train_preds):
    tmp = [0,0,0] # idx, cur, max
    delta = 0
    for tmp[0] in tqdm(np.arange(0.1, 0.501, 0.01)):
        tmp[1] = f1_score(train_y, np.array(train_preds)>tmp[0],average='micro')
        if tmp[1] > tmp[2]:
            delta = tmp[0]
            tmp[2] = tmp[1]
    print('best threshold is {:.4f} with F1 score: {:.4f}'.format(delta, tmp[2]))
    return delta

In [104]:
# Load Train-dataset
new_dataset_delta = data.TabularDataset(
                                        path = 'input/train.json',
                                        format = 'json',
                                        fields = fields
)

BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

new_iterator_delta = data.Iterator(
    dataset=new_dataset_delta, 
    batch_size=BATCH_SIZE,
    device=device,
    sort_key= lambda x: len(x.t),
    #sort_key = None,
    train=False,
    sort=False
)

In [106]:
preds_prob = predict_prob(model, new_iterator_delta)
preds_prob = np.asarray(preds_prob)


/home/max/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [101]:
# Load real Y values

train_delta = pd.read_json('input/train.json', orient='records', lines=True)
# Ground truth, from pandas DataFrame
# Each is a 126 item list in a DataFrame
# Trun into 2-dim np.array

# explode the list into columns
temp = train_delta['codes'].apply(pd.Series)
# turn to np.array
y_true_delta = np.asarray(temp)

In [109]:
#from tqdm import tqdm_notebook, tnrange
#from tqdm.auto import tqdm
from tqdm import *

# Find best threshold
delta = bestThresshold(y_true_delta,preds_prob)


100%|██████████| 41/41 [01:11<00:00,  1.74s/it]


best threshold is 0.3400 with F1 score: 0.8888


#### we got delta, now use it for test-set

In [110]:
preds_prob = predict_prob(model, new_iterator1)
preds_prob = np.asarray(preds_prob)

/home/max/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [118]:
preds_final = (preds_prob > delta).astype(int)

In [119]:
preds_pd = pd.DataFrame(preds_prob)
preds_pd.to_csv('output/predictions_trainset_test.csv', header=False, index=False)
test = pd.read_json('input/test.json', orient='records', lines=True)

In [120]:
from sklearn.metrics import f1_score

f1_score(y_true,
         preds_final,
         average='micro')

0.8859053316910083

### Predict on new data

In [149]:
# Save data as JSON
new_data = reuters = pd.read_pickle('input/data_new.pkl')
new_data.to_json('input/new_data.json', orient='records', lines=True)
len(new_data)

33142

In [129]:
temp = pd.read_json('input/new_data.json', orient='records', lines=True)

In [130]:
temp.head()

,codes,headline,text
0,[],PRESS DIGEST - SOUTH AFRICA - APRIL 10.,\nThese are the leading stories in the South A...
1,[],OFFICIAL JOURNAL CONTENTS - OJ C 110 OF APRIL ...,\n*\n(Note - contents are displayed in reverse...
2,[],OFFICIAL JOURNAL CONTENTS - OJ L 94 OF APRIL 9...,\n*\n(Note - contents are displayed in reverse...
3,[],OFFICIAL JOURNAL CONTENTS - OJ C 55 OF FEBRUAR...,"\n*\nMinutes of the sitting of Wednesday, 29 J..."
4,[],Toronto stocks close easier in lackluster deal...,\nCHANGE\t\t\t\t CHANGE\nTSE\t 5790.11 ...


In [131]:
fields_predict = {'headline': ('h', HEADLINE), 'text': ('t', TEXT)} # 'codes': ('l', LABELS)
fields_predict

{'headline': ('h', <torchtext.data.field.Field at 0x7fe5c90ce7b8>),
 'text': ('t', <torchtext.data.field.Field at 0x7fe5c90ce6d8>)}

In [150]:
# New data dataset
new_dataset = data.TabularDataset(
                                        path = 'input/new_data.json',
                                        format = 'json',
                                        fields = fields_predict
                                        #fields = fields
)

BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

new_iterator = data.Iterator(
    dataset=new_dataset, 
    batch_size=BATCH_SIZE,
    device=device,
    sort_key= lambda x: len(x.t),
    train=False,
    #sort=None
    sort=False
)

In [151]:
preds_prob = predict_prob(model, new_iterator)
preds_prob = np.asarray(preds_prob)

/home/max/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [152]:
preds_final = (preds_prob > delta).astype(int)

In [153]:
# Turn it to Pandas for using pandas .to_csv
preds_pd = pd.DataFrame(preds_final)

In [154]:
#datetime_string = datetime.now().strftime("%Y%m%d-%H%M")
#print(datetime_string)
preds_pd.to_csv('output/predictions_newdata_fulltraindata_CNN.csv', header=False, index=False)

In [155]:
len(preds_pd)

33142

In [156]:
preds_pd.head(5)

,0,1,2,3,4,5,6,7,8,9,...,116,117,118,119,120,121,122,123,124,125
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


### Write predictions to file

In [157]:
datetime_string = datetime.now().strftime("%Y%m%d-%H%M")
print(datetime_string)

20190218-1616


In [158]:
preds_pd.to_csv('output/predictions'+datetime_string+'.csv', header=False, index=False)